In [1]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import bisect

torch.manual_seed(1337)

n_features = 1

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 33 # what is the maximum context length for predictions?
max_iters = 5000
epochs = 3
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
st_dims = 8
ac_dims = 1
rw_dims = 1
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
vocab = 100

/Users/guevara/opt/anaconda3/envs/trajectory/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cols = ["Unnamed: 0", "X Coordinate", "Y Coordinate", "Linear Velocity X", "Linear Velocity Y", "Angle", \
       "Angular Velocity", "Leg1", "Leg2", "index"]
cols2 = ["Unnamed: 0", "observations", "next_observations", "index"]
obs_cols = ["X Coordinate", "Y Coordinate", "Linear Velocity X", "Linear Velocity Y", "Angle", \
       "Angular Velocity", "Leg1", "Leg2", "actions", "rewards", "rewardstg"]
renames = {'Action':'actions', 'Reward':'rewards', 'observation_x':'observations', 'observation_y':'next_observations'}

In [3]:

def get_data():
    dataframe = pd.read_csv("medium_play.csv")
    dataframe['observation'] = list(dataframe.iloc[:, 1:9].values)
    observations = dataframe['observation'].copy()
    observations.drop(observations.index[0], inplace=True)
    dataframe.drop(dataframe.index[-1], inplace=True)
    observations = observations.reset_index()
    merged_data = dataframe.merge(observations, left_index=True, right_index=True)
    merged_data["terminals"] = False
    merged_data.loc[184753:, "Episode"] += 2000
    grouped_episodes = merged_data.groupby("Episode")
    merged_data = merged_data.rename(columns=renames)
    for episode, group in grouped_episodes:
        last_index = group.index[-1]  # Get the index of the last row in the group
        merged_data.loc[last_index, 'terminals'] = True
        every = group.index
        merged_data.loc[every, 'rewardstg'] = merged_data.loc[every, 'rewards'].cumsum()
        merged_data.loc[every, 'rewardstg'] -= merged_data.loc[every[-1], 'rewardstg']
        merged_data.loc[every, 'rewardstg'] *= -1
    observations = np.vstack(merged_data['observations'])
    next_observations = np.vstack(merged_data['next_observations'])
    actions = torch.tensor(merged_data["actions"].values, dtype=torch.long)
    rewards = torch.tensor(merged_data["rewardstg"].values, dtype=torch.float)
    observations = torch.tensor(observations, dtype=torch.float)
    next_observations = torch.tensor(next_observations)
    merged_data.drop(cols2, inplace=True, axis=1)
    return merged_data, actions, rewards, observations


data, actions, rewards, observations = get_data()



In [4]:
#this class requres SAR sequence
class UniformDiscretizer():
    def __init__(self, dataframe, vocab):
        self.columns = dataframe.columns
        self.mins = []
        self.maxs = []
        self.steps = []
        self.offsets = []
        self.data = dataframe
        self.vocab = vocab
        self.total_vocab = 0

    def fit_transform(self):
        discretized, targets = pd.DataFrame(), pd.DataFrame()
        t_vocab = 0

        for i, col in enumerate(self.columns):
            self.offsets.append(i*100)

            mini, maxi = self.data[col].min(), self.data[col].max()

            step = (maxi - mini)/(vocab-1)

            targets[col] = ((data[col] - mini) / step).astype(int)

            discretized[col] = self.offsets[-1] + targets[col]

            t_vocab += targets[col].nunique()

            self.mins.append(mini), self.maxs.append(maxi), self.steps.append(step)

        self.offsets.append(self.offsets[-1] + 100)

        self.total_vocab = self.offsets[-1]

        return discretized, targets

    def raw_to_target(self, raw, i):
        return int((raw - self.mins[i]) / self.steps[i])

    def raw_to_discretized(self, raw, i):
        tok = self.raw_to_target(raw, i)
        return tok + self.offsets[i]

    def trajectory_to_discretized(self, trajectory):
        return trajectory + self.offsets[:-1]

    def trajectory_to_target(self, trajectory):
        return trajectory - self.offsets[:-1]

    def token_to_discretized(self, token, column_idx):
        return token + self.offsets[column_idx]

    def token_to_target(self, token):
        column_idx = bisect.bisect_left(self.offsets, token) - 1
        return token - self.offsets[column_idx]

    def discretized_recover(self, token):
        column_idx = bisect.bisect_left(self.offsets, token) - 1
        token = token - self.offsets[column_idx]
        return self.target_recover(token, column_idx)

    def target_recover(self, token, column_idx):
        return (token)*self.steps[column_idx] + self.mins[column_idx]

In [5]:
train_df = data[obs_cols]
UD = UniformDiscretizer(train_df, 100)
discretized, targets = UD.fit_transform()
discretized

,X Coordinate,Y Coordinate,Linear Velocity X,Linear Velocity Y,Angle,Angular Velocity,Leg1,Leg2,actions,rewards,rewardstg
0,49,176,260,387,449,551,600,700,866,939,1040
1,50,177,260,386,449,550,600,700,899,940,1040
2,50,177,260,385,449,550,600,700,800,941,1040
3,50,177,260,385,449,550,600,700,866,940,1040
4,50,177,260,384,449,550,600,700,800,941,1040
...,...,...,...,...,...,...,...,...,...,...,...
400088,45,117,250,377,449,551,600,700,899,941,1064
400089,45,117,250,377,449,551,600,700,899,941,1064
400090,45,117,250,377,449,551,600,700,899,941,1064
400091,45,117,249,377,449,551,698,700,833,946,1063


In [130]:
discretized = torch.tensor(discretized.values, device=device, dtype=torch.long).reshape(-1)
targets = torch.tensor(targets.values, device=device, dtype=torch.long).reshape(-1)
total_vocab = discretized.max()+1
trajectory_length = len(UD.columns)

In [131]:
train_df

,X Coordinate,Y Coordinate,Linear Velocity X,Linear Velocity Y,Angle,Angular Velocity,Leg1,Leg2,actions,rewards,rewardstg
0,0.009168,1.422375,0.466317,0.271966,-0.010169,-9.798397e-02,0.0,0.0,2.0,-3.524214,-203.813656
1,0.013870,1.427906,0.477506,0.245732,-0.017304,-1.427123e-01,0.0,0.0,3.0,-1.019767,-202.793890
2,0.018573,1.432837,0.477528,0.219063,-0.024437,-1.426711e-01,0.0,0.0,0.0,-0.046968,-202.746922
3,0.023168,1.438152,0.467235,0.236097,-0.032168,-1.546410e-01,0.0,0.0,2.0,-1.423334,-201.323588
4,0.027762,1.442868,0.467257,0.209417,-0.039898,-1.546060e-01,0.0,0.0,0.0,-0.106747,-201.216841
...,...,...,...,...,...,...,...,...,...,...,...
400088,-0.087697,-0.000911,0.006983,0.000119,0.000588,-3.050588e-04,0.0,0.0,3.0,-0.650156,10.126583
400089,-0.087571,-0.000910,0.012630,0.000002,0.000589,3.889892e-07,0.0,0.0,3.0,-0.581971,10.708553
400090,-0.087386,-0.000911,0.018467,-0.000002,0.000588,-5.688639e-07,0.0,0.0,3.0,-0.595277,11.303831
400091,-0.087375,-0.000908,0.001081,0.000118,0.000601,2.431244e-04,1.0,0.0,1.0,11.707871,-0.404041


In [42]:
UD.raw_to_discretized(-0.650156, 9)

941

In [15]:
device

'cpu'

In [16]:
def get_batch():
    ix = torch.randint(len(discretized) - block_size, (batch_size,))
    x = torch.stack([discretized[i:i+block_size] for i in ix])
    y = torch.stack([targets[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    tt.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        x, y = get_batch()
        logits, loss = tt(x, targets=y)
        losses[k] = loss.item()
    out = losses.mean()
    tt.train()
    return out

In [17]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(3*block_size, 3*block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [18]:
# super simple bigram model
class TrajectoryTransformer(nn.Module):

    def __init__(self):
        super().__init__()
        self.init_embd = nn.Embedding(total_vocab, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab)

    def forward(self, sequence, targets=None):
        vocab_embedding = self.init_embd(sequence)#(B,T,C)
        B, T, C = vocab_embedding.shape
        pos_encoding = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = vocab_embedding + pos_encoding #(B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)

        preds = self.lm_head(x) # (B,T,vocab)

        if targets is None:
            loss = None
        else:
            B, T, C = preds.shape
            preds = preds.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(preds, targets)

        return preds, loss

In [81]:
tt = TrajectoryTransformer()
tt.to(device)
optimizer = torch.optim.AdamW(tt.parameters(), lr=learning_rate)

In [87]:
for _ in range(epochs):
    for iteration in range(101):
      # every once in a while evaluate the loss on train and val sets
        if iteration % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iteration}: train loss {losses:.4f}")

        # sample a batch of data
        x, y = get_batch()

        # evaluate the loss
        logits, loss = tt(x, targets=y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

step 0: train loss 0.8725
step 100: train loss 0.8695
step 0: train loss 0.8774
step 100: train loss 0.8641
step 0: train loss 0.8534
step 100: train loss 0.8719


In [84]:
save_path = 'tt_instance_1.model'

# Save the model's state dictionary to the file
torch.save(tt.state_dict(), save_path)

In [85]:
state_dict = torch.load(save_path)
tt.load_state_dict(state_dict)

<All keys matched successfully>

In [23]:
context = discretized.view(-1, trajectory_length)[4200]

In [30]:
type([])

list

In [140]:
import torch
import numpy as np

def beam_search(model, context, st_width=3, ac_width=4):
    if type(context) != list:
        context = context.detach().cpu().numpy()
    window_by_index = [st_width] * st_dims + [ac_width] * ac_dims + [1] + [1]
    top_beams_context = [context]
    top_beams_prob = [0]

    for i in range(trajectory_length):
        square_beams_context = []
        square_beams_prob = []
        k = window_by_index[i]

        for seq, prob in zip(top_beams_context, top_beams_prob):
            logits, _ = model(torch.tensor(seq[-block_size:], device=device).unsqueeze(0))
            logits = F.softmax(logits[0, -1, :], dim=0).detach().cpu().numpy()

            top_k = np.argpartition(logits, -k)[-k:]
            square_beams_prob.extend(logits[top_k] + prob)

            top_dis_k = [UD.token_to_discretized(tk, i) for tk in top_k]
            square_beams_context.extend([np.concatenate([seq, tk.reshape(1)]) \
                                  for tk in top_dis_k])

        if i < st_dims:
            idxs = np.argsort(square_beams_prob)[-k:]
        else:
            idxs = np.argsort(square_beams_prob)

        top_beams_context = np.array(square_beams_context)[idxs]
        top_beams_prob = np.array(square_beams_prob)[idxs]

    rwds = [UD.discretized_recover(x[-2]) + UD.discretized_recover(x[-1])
                                  for x in top_beams_context]
    rwds_idx = np.argsort(rwds)
    top_beams_context = top_beams_context[rwds_idx]
    top_beams_prob = top_beams_prob[rwds_idx]

    return top_beams_context, top_beams_prob, rwds

def extract_action(trajectory):
    return trajectory[st_dims::trajectory_length]


In [154]:
print(context)
top_beams_context,  top_beams_prob, rwds = beam_search(tt, context, 100, 4)
# temp = top_beams_context[-1]
# print(temp)
print(sorted(rwds))

tensor([  46,  175,  234,  373,  450,  552,  600,  700,  866,  941, 1045])
[-164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, -164.92742893369225, 

# Test

In [142]:
import gymnasium as gym

In [164]:
env = gym.make("LunarLander-v2")


def testing(model, context):
    total_reward = 0.0
    desired_reward = 200

    state, _ = env.reset(seed=None)
    new_state, reward, terminated, _, _ = env.step(0)
    terminated = False
    time = 0
    to_encode = list(state) + [0] + [reward] + [desired_reward]
    context = [UD.raw_to_discretized(r, i) for i, r in enumerate(to_encode)]
    while not terminated and time <= 500:
#         print(context)
        seqs, probs, rwds = beam_search(tt, context)
        print(sorted(rwds))

        action_tokens = [UD.token_to_target(i,) for i in extract_action(seqs[-1])[1:]]
        for action_token in action_tokens:
            action = int(UD.discretized_recover(action_token))
            state, reward, terminated, _, _ = env.step(action)

            total_reward += reward
            desired_reward -= reward
            to_encode = list(state) + [action] + [reward] + [desired_reward]
            context = [UD.raw_to_discretized(r, i) for i, r in enumerate(to_encode)]
        
        time += 1
        if time%20 == 0:
            print(time)

      # Print reward
    print("total_reward = {}".format(total_reward))
    env.close()

In [166]:
testing(tt, context)

[188.829535199558, 188.829535199558, 188.829535199558, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181]
[188.829535199558, 188.829535199558, 188.829535199558, 188.829535199558, 188.829535199558, 188.829535199558, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181, 191.2486332314181]
[197.73393675368584, 197.73393675368584, 197.73393675368584, 197.73393675368584, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594]
[197.73393675368584, 197.73393675368584, 197.73393675368584, 197.73393675368584, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594, 200.15303478554594]
[197.73393675368584, 197.73393675368584, 197.73393675368584, 197.7339

[233.35154297019676, 233.35154297019676, 233.35154297019676, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 238.18973903391696, 238.18973903391696, 238.18973903391696]
[233.35154297019676, 233.35154297019676, 233.35154297019676, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 238.18973903391696, 238.18973903391696, 238.18973903391696]
[233.35154297019676, 233.35154297019676, 233.35154297019676, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 238.18973903391696, 238.18973903391696, 238.18973903391696]
[233.35154297019676, 233.35154297019676, 233.35154297019676, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 235.77064100205686, 238.18973903391696, 238.18973903391696, 247.0941405880448]
40
[233.35154297019676, 233.351542970